In [107]:
import pandas as pd
import sqlite3
from Levenshtein import distance
from fuzzywuzzy import fuzz

# connect to the SQLite database and read the data into a pandas dataframe
conn = sqlite3.connect('data_files/words.db')
df = pd.read_sql('SELECT * FROM words', conn)

# close the database connection
conn.close()

In [108]:
#df['word'] = df['word'].str.split(',').str[0]
df['Length_word'] = df['word'].str.len()
#df['translation'] = df['translation'].str.split(',').str[0]
df['Length_translation'] = df['translation'].str.len()
# Calculate Levenshtein Distance between word and translation
#df['Distance'] = df.apply(lambda row: distance(row['word'], row['translation']), axis=1)
df['Similarity'] = df.apply(lambda row: fuzz.ratio(row['word'], row['translation']), axis=1)

In [109]:
# Calculate word lengths and divide into three bins
#df['Difficulty'] = pd.cut(df['Length'], bins=3, labels=['Easy', 'Medium', 'Hard'])

In [110]:
# dividing into three quantiles
df['d1'] = pd.qcut(df['Length_word'], q=2, labels=[0, 1])
df['d1'] = df['d1'].astype(int)
df['d2'] = pd.qcut(df['Length_translation'], q=2, labels=[0, 1])
df['d2'] = df['d2'].astype(int)
df['d3'] = pd.qcut(df['Similarity'], q=3, labels=[2, 1, 0])
df['d3'] = df['d3'].astype(int)

In [111]:
df['difficulty'] = df['d1'] + df['d2'] + df['d3']
df.loc[df['Similarity'] >= 90, 'difficulty'] = 0

df = df.loc[:, ['word', 'type', 'translation', 'example', 'example_translation',
       'russian', 'appear', 'trial_d', 'trial_r', 'success', 'weight', 'Similarity', 'difficulty']]

In [41]:
writer = pd.ExcelWriter('data_files/test.xlsx')
df.to_excel(writer, sheet_name='update')
writer.close()

In [112]:
conn2 = sqlite3.connect('data_files/lessons.db')
lesson = pd.read_sql('SELECT * FROM lessons', conn2)
conn2.close()

In [113]:
lesson_df = lesson[lesson['known'] != 25]
lesson_df = lesson_df.loc[:, ['r', 'list_of_words', 'points']]


In [114]:
# Create a sample difficulty DataFrame
difficulty_df = df.loc[:,['word', 'difficulty']]

# Split 'list of words' column and retrieve difficulties
lesson_df['words'] = lesson_df['list_of_words'].str.split(';')
lesson_df['words'] = lesson_df['words'].apply(lambda x: [word.strip() for word in x])
lesson_df['dif'] = lesson_df['words'].apply(lambda x: sum(difficulty_df[difficulty_df['word'].isin(x)]['difficulty']))

In [115]:
# Remove the intermediate 'words' column
lesson_df.drop('words', axis=1, inplace=True)

In [116]:
writer = pd.ExcelWriter('data_files/test_check.xlsx')
lesson_df.to_excel(writer, sheet_name='update')
writer.close()

In [121]:
# Add a unique key column based on index
df['key'] = df.index

# Define the desired distribution of words by difficulty
difficulty_distribution = {4: 5, 3: 5, 2: 5, 1: 5, 0: 5}

# Sample words from each difficulty level based on difficulty and weight
selected_words = []
for difficulty, count in difficulty_distribution.items():
    words_subset = df[(df['difficulty'] == difficulty)]
    words_subset = words_subset.sample(n=count, weights=words_subset['weight'])
    selected_words.append(words_subset)
    
    
# Concatenate the selected words into a new DataFrame
new_words_df = pd.concat(selected_words)

writer = pd.ExcelWriter('data_files/new.xlsx')
new_words_df.to_excel(writer, sheet_name='update')
writer.close()
    
# Reset the index of the new DataFrame
new_words_df.reset_index(drop=True, inplace=True)

# Update the parameters of the selected words
new_words_df['weight'] = new_words_df['weight'] * 2  # Update the weight parameter as an example

# Merge the updated words with the original DataFrame based on the key column
df = pd.merge(df, new_words_df, on='key', how='left')

# Fill missing values in the original DataFrame with the updated values
df['difficulty_y'].fillna(df['difficulty_x'], inplace=True)
df['weight_y'].fillna(df['weight_x'], inplace=True)

# Drop redundant columns from the merged DataFrame
df.drop(['difficulty_x', 'weight_x', 'key'], axis=1, inplace=True)
df.rename(columns={'difficulty_y': 'difficulty', 'weight_y': 'weight'}, inplace=True)

In [122]:
writer = pd.ExcelWriter('data_files/final.xlsx')
df.to_excel(writer, sheet_name='update')
writer.close()